# Locality Sensitive Hashing

In [1]:
import random
import os
import numpy as np
import itertools
from random import randrange, shuffle
from shutil import copyfile
import collections

### Two sets of documents are used in this project: 
#### 1. Sample documents: documents that contain possibly plagiarised material
#### 2. Original documents: excerpts from original Wikipedia articles


### STEP - 1: Shingling 

#### Storing all sample documents' contents in a list of [(doc name, doc_contents)]:

In [5]:
os.chdir(<path to local directory that contains all the sample documents>)
data_sample = []
for file in os.listdir():
    with open(file, encoding="utf8", errors='ignore') as f:
        data_sample.append((file,f.read().lower()))
    

#### Storing all original documents' contents in a list of [(doc name, doc_contents)]:

In [6]:
os.chdir(<path to local directory that contains all the original documents>)
original_sample = []
for file in os.listdir():
    with open(file, encoding="utf8", errors='ignore') as f:
        original_sample.append((file,f.read().lower()))

#### Basic text cleaning - converting to lower case and removing all punctuations and special characters:

In [7]:
def clean_string(text):
    for ch in ['\n','*','_','{','}','[',']','(',')','<','>','#','+','-','.','!','$','\\',',','?','"','|','/','=']:
        if ch in text:
            text = text.replace(ch," ").strip()
    return text

In [8]:
clean_data_sample = [(x[0],clean_string(x[1])) for x in data_sample]

In [9]:
clean_original_sample = [(x[0],clean_string(x[1])) for x in original_sample]

#### Function for constructing n-shingles from the documents:

In [10]:
def get_shingles(size,doc):
    shingle_dict = {}
    for i in range (0,len(doc)):
        doc_shing = []
        lenth = len(doc[i][1])
        docid = doc[i][0]
        # Tokenizing text to aid in the formation of shingles
        tokens = doc[i][1].split()
        for j in range (0,lenth-size+1):
            sh = tuple(tokens[j:j+size])
            if sh not in doc_shing:
                doc_shing.append(sh)
        # Final shingle dictionary: {"docid":[shingles]}        
        shingle_dict[docid] = doc_shing
    return shingle_dict


#### Constructing 3-, 4- and 5-shingles and consolidating a list of these shingles from both the sample documents and the original documents:

In [11]:
shing_data_3 = set(list(itertools.chain(*list(get_shingles(3, clean_data_sample).values()))))
shing_orig_3 = set(list(itertools.chain(*list(get_shingles(3, clean_original_sample).values()))))
shing_3 = shing_data_3 | shing_orig_3
length_3 = len(shing_3)
# Total number of 3-shingles in both sample and original data files
length_3

7852

In [12]:
shing_data_4 = set(list(itertools.chain(*list(get_shingles(4, clean_data_sample).values()))))
shing_orig_4 = set(list(itertools.chain(*list(get_shingles(4, clean_original_sample).values()))))
shing_4 = shing_data_4 | shing_orig_4
length_4 = len(shing_4)
# Total number of 4-shingles in both sample and original data files
length_4

8786

In [13]:
shing_data_5 = set(list(itertools.chain(*list(get_shingles(5, clean_data_sample).values()))))
shing_orig_5 = set(list(itertools.chain(*list(get_shingles(5, clean_original_sample).values()))))
shing_5 = shing_data_5 | shing_orig_5
length_5 = len(shing_5)
# Total number of 5-shingles in both sample and original data files
length_5

9266

#### All sample documents and their constituent 5-shingles

In [14]:
shing_5_docs = get_shingles(5, clean_data_sample)
shing_5_docs = collections.OrderedDict(list(shing_5_docs.items()))
# Ordered dictionary for referencing documents and their shingles by index
list(shing_5_docs.items())[0][1][0:5]

[('bayes', 'theorem', 'is', 'an', 'important'),
 ('theorem', 'is', 'an', 'important', 'theorem'),
 ('is', 'an', 'important', 'theorem', 'relating'),
 ('an', 'important', 'theorem', 'relating', 'conditional'),
 ('important', 'theorem', 'relating', 'conditional', 'probabilities')]

Number of unique k-shingles for k={3,4,5}:
1. k=3: 7852
2. k=4: 8786
3. k=5: 9266

In [15]:
shing_5_list = list(shing_5)
shing_5_ind = {k: v for v, k in enumerate(shing_5_list)}
# Creating a dictionary of indexes for all 5-shingles
list(shing_5_ind.items())[0:5]

[(('it', 'inherited', 'from', 'the', 'peropos'), 0),
 (('a', 'it', 'is', 'previous', 'in'), 1),
 (('person', 'may', 'be', 'seen', 'to'), 2),
 (('be', 'written', '7', 'so', 'this'), 3),
 (('the', 'philosophy', 'of', 'science', 'it'), 4)]

### STEP - 2: Min-Hashing

In [16]:
# Function to generate hashing functions using the (ax + b) mod N formula, 
# Parameters:
# 1. N: the total number of 5-shingles in the sample and original documents combined.
# 2. L: number of hash functions
# Returns the generated hash functions: L new lists of size N
def get_hash_functions(N,L):
    hash_functions = []
    
    for itr in range(L):
        # a and b are random numbers
        a=randrange(1,400)
        b=randrange(1,400)
        
        new_hash_function = []
        for i in range(N):
            # Implementing the (ax + b) mod N formula:
            new_hash_function.append((a * i + b) % N)
        
        hash_functions.append(new_hash_function)
    return hash_functions
        

In [17]:
# Generating hash functions for only shingle index created for k=5
#L = {50,100,200,500,1000}
N = length_5
hash_func50 = get_hash_functions(N, 50)
hash_func100 = get_hash_functions(N, 100)
hash_func200 = get_hash_functions(N, 200)
hash_func500 = get_hash_functions(N, 500)
hash_func1000 = get_hash_functions(N, 1000)

#### Function for composing the signature matrix. Spits out the signature matrix in transposed form:

In [18]:
# Calculates minhash values for each document, over all hash documents, avoiding the need for an input matrix
# Parameters:
# 1. List of hash functions
# 2. Number of hash functions
# 3. Dictionary of 5-shingles from sample documents
# 4. Dictionary of indexes of all shingles
# Returns signature matrix of dimensions: rows(# of documents) x columns(# of hash functions used, minhash values)
def get_sig_m(hash_f, L, sh_5, sh_5_index):
    sig_m = []
    i = 0
    # Iterating through sample documents:
    for k in sh_5.values():
        sig_list = list(np.ones(L) * np.inf)
        # Iterating through all shingles:
        for sh, v in sh_5_index.items():
            # Where a shingle from the dictionary of indexes is found in the document, determine
            # the minshash value
            if sh in k:
                for j in range(L):
                    if hash_f[j][v] < sig_list[j]:
                        sig_list[j] = hash_f[j][v]
        i = i + 1
        sig_m.append(sig_list)
    return np.array(sig_m)

In [19]:
signature_50 = get_sig_m(hash_func50,50, shing_5_docs,shing_5_ind)
signature_100 = get_sig_m(hash_func100,100, shing_5_docs,shing_5_ind)
signature_200 = get_sig_m(hash_func200,200, shing_5_docs,shing_5_ind)
signature_500 = get_sig_m(hash_func500,500, shing_5_docs,shing_5_ind)
signature_1000 = get_sig_m(hash_func1000,1000,shing_5_docs,shing_5_ind)

In [20]:
# Fact-checking with one file from the original sample directory
original = 'Original_Sample/orig_taskc.txt'
# Jaccard similarity threshold
t = 0.20

#### Step 1: All original documents and their constituent 5-shingles

In [21]:
sh_5_orig = get_shingles(5, clean_original_sample)
sh_5_orig = collections.OrderedDict(sh_5_orig)
# Creating a dictionary of indexes for all 5-shingles
list(sh_5_orig.items())[0][1][0:5]

[('in', 'probability', 'theory', "bayes'", 'theorem'),
 ('probability', 'theory', "bayes'", 'theorem', 'often'),
 ('theory', "bayes'", 'theorem', 'often', 'called'),
 ("bayes'", 'theorem', 'often', 'called', "bayes'"),
 ('theorem', 'often', 'called', "bayes'", 'law')]

#### Calculating the jaccard distance: intersection/union

In [22]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

Function for calculating the similarity of documents:

In [23]:
# Function that gauges similarity of documents by calculating the jaccard similarity of 
# sets of minhash values of documents
# Parameters:
# 1. (Transposed) signature matrix of sample documents
# 2. (Transposed) signature matrix of original documents
# 3. Threshold
# Returns a list of sample-original document pairs with a jaccard similarity greater than the supplied threshold

# List of the shingles from the sample documents and the original documents
da = list(shing_5_docs.items())
ori = list(sh_5_orig.items())
def sim_docs(sign_samp, sign_orig, t):
    sim = collections.defaultdict()
    # Calculating the jaccard similarities of all sample documents against all original documents
    for i in range(0,len(sign_samp)):
        for j in range(0,len(sign_orig)):
            card = jaccard(sign_samp[i].tolist(),sign_orig[j].tolist())
            # If the similarity score is greater than the threshold, output
            if card >= t:
                d = da[i][0]
                o = ori[j][0]
                k = d+" "+o
                sim[d+" & "+o] = card
    return sorted(sim.items(), key=lambda k_v: k_v[1], reverse=True)

#### Step 2: Computing signature matrices for original documents with a varied number of hash functions:

In [24]:
signature_50_orig = get_sig_m(hash_func50,50,sh_5_orig,shing_5_ind)
signature_100_orig = get_sig_m(hash_func100,100,sh_5_orig,shing_5_ind)
signature_200_orig = get_sig_m(hash_func200,200,sh_5_orig,shing_5_ind)
signature_500_orig = get_sig_m(hash_func500,500,sh_5_orig,shing_5_ind)
signature_1000_orig = get_sig_m(hash_func1000,1000,sh_5_orig,shing_5_ind)

### For each L = {50,100,200,500,1000}, all documents listed under have Jaccard similarity > t=0.2, and are sorted in decreasing order of Jaccard similarity

#### Step 3: Documents that have a similarity greater than the threshold:

In [25]:
sim_docs(signature_50, signature_50_orig, 0.2)[0:5]

[('g0pE_taska.txt & orig_taska.txt', 0.5625),
 ('g4pC_taska.txt & orig_taska.txt', 0.5384615384615384),
 ('g3pA_taskd.txt & orig_taskd.txt', 0.47058823529411764),
 ('g4pC_taskd.txt & orig_taskd.txt', 0.4084507042253521),
 ('g0pD_taska.txt & orig_taska.txt', 0.2987012987012987)]

In [26]:
sim_docs(signature_100, signature_100_orig, 0.2)[0:5]

[('g0pE_taska.txt & orig_taska.txt', 0.3888888888888889),
 ('g4pC_taska.txt & orig_taska.txt', 0.36054421768707484),
 ('g3pA_taskd.txt & orig_taskd.txt', 0.2987012987012987),
 ('g0pC_taska.txt & orig_taska.txt', 0.27388535031847133),
 ('g2pC_taska.txt & orig_taska.txt', 0.26582278481012656)]

In [27]:
sim_docs(signature_200, signature_200_orig, 0.2)[0:5]

[('g3pA_taskd.txt & orig_taskd.txt', 0.2345679012345679),
 ('g0pE_taska.txt & orig_taska.txt', 0.21212121212121213),
 ('g4pC_taska.txt & orig_taska.txt', 0.20481927710843373),
 ('g1pB_taska.txt & orig_taskd.txt', 0.2012012012012012)]

In [28]:
sim_docs(signature_500, signature_500_orig, 0.1)[0:5]

[('g0pE_taska.txt & orig_taska.txt', 0.11234705228031146),
 ('g4pC_taska.txt & orig_taska.txt', 0.10741971207087486),
 ('g0pE_taske.txt & orig_taska.txt', 0.10619469026548672),
 ('g1pD_taskd.txt & orig_taska.txt', 0.10619469026548672),
 ('g0pE_taskd.txt & orig_taska.txt', 0.10497237569060773)]

In [29]:
sim_docs(signature_1000, signature_1000_orig, 0.05)[0:5]

[('g0pE_taskd.txt & orig_taska.txt', 0.06553010122535961),
 ('g3pC_taskd.txt & orig_taska.txt', 0.06496272630457935),
 ('g0pE_taske.txt & orig_taska.txt', 0.06439595529536987),
 ('g3pB_taske.txt & orig_taska.txt', 0.06439595529536987),
 ('g0pD_taske.txt & orig_taska.txt', 0.06439595529536987)]

### STEP - 3: LSH 

In [30]:
# Next, to hash signature matrix into B buckets
# Using the technique of splitting the signature matrix into b bands of r rows
# Converting only the signature matrix generated with L=1000
b = 50
r = 20
B = 199

#### Segregating the signature function into b bands of r rows each:

In [31]:
# Function takes the signature matrix in transposed form, and splits every document's 
# minhash values into b bands of r rows
# Parameters:
# 1. Signature matrix in transposed form
# 2. Number of bands
# 3. Number of rows in each band
# Returns a matrix of separated bands
def bands(sig,b,r):
    l = len(sig[0])
    x_banded = []
    for doc in sig:
        x_bands = []
        n = 0
        while n <= l-r:
            x_bands.append(tuple(doc[n:n+r]))
            n = n+r
        x_banded.append(x_bands)
    return np.array(x_banded)

#### Function to hash the values in the b bands:

In [32]:
# Function hashes the minhashes values using the formula: (sum over band(minhash value*random integer a)) % N
# Parameters:
# 1. Banded matrix from previous step
# 2. Number of buckets, N
def hash_vals(band_mat, N):
    # Generating a list of random integers between 1 and 20 that will be used across bands
    a = list(range(1,r+1))
    shuffle(a)
    lsh = []
    for doc in band_mat:
        hashed = []
        for v in doc:
            # Hashing the minhash values into buckets
            summed = sum([v[x]*a[x]for x in range(r)])%N
            hashed.append(summed)
        lsh.append(hashed)
    return np.array(lsh)

#### Step 1: Banding and hashing the sample documents' and original documents' minhash signatures

In [33]:
banded_mat = bands(signature_1000,b,r)
hashed_docs = hash_vals(banded_mat,B)
banded_mat_orig = bands(signature_1000_orig,b,r)
hashed_docs_orig = hash_vals(banded_mat_orig,B)
# One document from the set of original documents available
test_orig = hashed_docs_orig[1].tolist()
# Name of the aforementioned original document
orig_name = list(sh_5_orig.items())[1][0]

#### Step 2: Function for Locality Sensitive Hashing:

In [34]:
# This function takes the banded and hashed matrices for the sample and original documents and 
# computes similarity based on the principal that similar documents are likely to end up in
# the same bucket
# Parameters:
# 1. Banded + hashed sample documents
# 2. Dictionary of sample documents and constiuent 5-shingles
# 3. Original banded + hashed document
# 4. Threshold
# Returns:
# 1. Candidates (at least one same hash value shared between sample document and original document)
# 2. False negatives
# 3. False positives
# 4. Jaccard similarity scores of candidate documents
def local_sh(hash_docs, sh_5, test_o,t):
    cand = []
    jaccard_s = []
    false_neg = []
    false_pos = []
    ind = 0
    for x in hash_docs:
        fl = 0
        # Name of the sample document
        doc = list(sh_5.items())[ind][0]
        # A document is considered a candidate only when at least one of its banded+hashed value
        # is the shared with the banded+hashed values of the original document
        if len([i for i, j in zip(x, test_o) if i == j]) >= 1:
            cand.append(doc)
            fl = 1
        j = jaccard(list(x),test_o)
        if fl > 0:
            # Jaccard similarity scores of all candidate documents
            jaccard_s.append((doc,orig_name,j))
        # False negatives are those documents whose similarity scores are greater than the
        # threshold, but are not indentified as candidates
        if j > t and doc not in cand:
            false_neg.append(doc)
        # False positives are those documents whose similarity scores are lower than the
        # threshold, but are indentified as candidates
        elif j <= t and doc in cand:
            false_pos.append(doc)
        ind = ind+1
    jaccard_s.sort(key=lambda x:x[2], reverse=True)
    return (cand, false_neg, false_pos, jaccard_s)


In [35]:
candidates, false_negatives, false_positives, jaccards = local_sh(hashed_docs,shing_5_docs,test_orig,0.2)

In [36]:
candidates

['g0pE_taskd.txt',
 'g1pA_taske.txt',
 'g1pA_taska.txt',
 'g0pB_taska.txt',
 'g0pB_taske.txt',
 'g4pE_taske.txt',
 'g0pA_taska.txt',
 'g2pB_taska.txt',
 'g4pE_taska.txt',
 'g2pE_taskd.txt',
 'g1pD_taska.txt',
 'g2pE_taske.txt',
 'g3pA_taskd.txt',
 'g4pB_taskd.txt',
 'g4pB_taske.txt',
 'g1pD_taske.txt']

In [37]:
len(candidates)

16

In [38]:
jaccards

[('g4pE_taske.txt', 'orig_taske.txt', 0.20481927710843373),
 ('g2pE_taske.txt', 'orig_taske.txt', 0.17647058823529413),
 ('g1pA_taske.txt', 'orig_taske.txt', 0.16279069767441862),
 ('g0pB_taska.txt', 'orig_taske.txt', 0.16279069767441862),
 ('g0pB_taske.txt', 'orig_taske.txt', 0.16279069767441862),
 ('g1pD_taska.txt', 'orig_taske.txt', 0.13636363636363635),
 ('g0pE_taskd.txt', 'orig_taske.txt', 0.12359550561797752),
 ('g1pA_taska.txt', 'orig_taske.txt', 0.12359550561797752),
 ('g2pE_taskd.txt', 'orig_taske.txt', 0.12359550561797752),
 ('g4pB_taskd.txt', 'orig_taske.txt', 0.12359550561797752),
 ('g4pE_taska.txt', 'orig_taske.txt', 0.1111111111111111),
 ('g4pB_taske.txt', 'orig_taske.txt', 0.0989010989010989),
 ('g2pB_taska.txt', 'orig_taske.txt', 0.08695652173913043),
 ('g3pA_taskd.txt', 'orig_taske.txt', 0.08695652173913043),
 ('g0pA_taska.txt', 'orig_taske.txt', 0.07526881720430108),
 ('g1pD_taske.txt', 'orig_taske.txt', 0.06382978723404255)]

In [39]:
false_positives

['g0pE_taskd.txt',
 'g1pA_taske.txt',
 'g1pA_taska.txt',
 'g0pB_taska.txt',
 'g0pB_taske.txt',
 'g0pA_taska.txt',
 'g2pB_taska.txt',
 'g4pE_taska.txt',
 'g2pE_taskd.txt',
 'g1pD_taska.txt',
 'g2pE_taske.txt',
 'g3pA_taskd.txt',
 'g4pB_taskd.txt',
 'g4pB_taske.txt',
 'g1pD_taske.txt']

In [40]:
len(false_positives)

15

In [41]:
false_negatives

[]

In [42]:
len(false_negatives)

0

#### Experimenting with different values of b, r, B to reduce the number of false positives and false negatives

In [43]:
b = 10
r = 100
B = 200
t = 0.2
banded_mat = bands(signature_1000,b,r)
hashed_docs = hash_vals(banded_mat,B)
banded_mat_orig = bands(signature_1000_orig,b,r)
hashed_docs_orig = hash_vals(banded_mat_orig,B)
test_orig = hashed_docs_orig[1].tolist()
orig_name = list(sh_5_orig.items())[1][0]

candidates, false_negatives, false_positives, jaccards = local_sh(hashed_docs,shing_5_docs,test_orig,t)
len_candidates = len(candidates)
len_fn = len(false_negatives)
len_fp = len(false_positives)
print(jaccards)
print("Candidates:",len_candidates,", False negatives:",len_fn,", False positives:",len_fp)

[('g3pB_taska.txt', 'orig_taske.txt', 0.1111111111111111), ('g0pE_taskd.txt', 'orig_taske.txt', 0.05263157894736842), ('g0pC_taska.txt', 'orig_taske.txt', 0.05263157894736842), ('g3pC_taskd.txt', 'orig_taske.txt', 0.05263157894736842)]
Candidates: 4 , False negatives: 0 , False positives: 4


In [44]:
b = 100
r = 10
B = 250
t = 0.2
banded_mat = bands(signature_1000,b,r)
hashed_docs = hash_vals(banded_mat,B)
banded_mat_orig = bands(signature_1000_orig,b,r)
hashed_docs_orig = hash_vals(banded_mat_orig,B)
test_orig = hashed_docs_orig[1].tolist()
orig_name = list(sh_5_orig.items())[1][0]

candidates, false_negatives, false_positives, jaccards = local_sh(hashed_docs,shing_5_docs,test_orig,t)
len_candidates = len(candidates)
len_fn = len(false_negatives)
len_fp = len(false_positives)
print(jaccards)
print("Candidates:",len_candidates,", False negatives:",len_fn,", False positives:",len_fp)

[('g1pB_taske.txt', 'orig_taske.txt', 0.19760479041916168), ('g0pC_taska.txt', 'orig_taske.txt', 0.19047619047619047), ('g0pC_taskd.txt', 'orig_taske.txt', 0.17647058823529413), ('g2pB_taskd.txt', 'orig_taske.txt', 0.17647058823529413), ('g2pC_taske.txt', 'orig_taske.txt', 0.17647058823529413), ('g1pD_taskd.txt', 'orig_taske.txt', 0.17647058823529413), ('g0pE_taske.txt', 'orig_taske.txt', 0.16279069767441862), ('g3pB_taske.txt', 'orig_taske.txt', 0.16279069767441862), ('g0pD_taska.txt', 'orig_taske.txt', 0.16279069767441862), ('g1pA_taskd.txt', 'orig_taske.txt', 0.15606936416184972), ('g4pC_taske.txt', 'orig_taske.txt', 0.15606936416184972), ('g2pE_taske.txt', 'orig_taske.txt', 0.15606936416184972), ('g2pB_taska.txt', 'orig_taske.txt', 0.14942528735632185), ('g0pB_taske.txt', 'orig_taske.txt', 0.14285714285714285), ('g0pD_taskd.txt', 'orig_taske.txt', 0.12359550561797752)]
Candidates: 15 , False negatives: 1 , False positives: 15


In [45]:
b = 2
r = 500
B = 500
t = 0.3
banded_mat = bands(signature_1000,b,r)
hashed_docs = hash_vals(banded_mat,B)
banded_mat_orig = bands(signature_1000_orig,b,r)
hashed_docs_orig = hash_vals(banded_mat_orig,B)
test_orig = hashed_docs_orig[1].tolist()
orig_name = list(sh_5_orig.items())[1][0]

candidates, false_negatives, false_positives, jaccards = local_sh(hashed_docs,shing_5_docs,test_orig,t)
len_candidates = len(candidates)
len_fn = len(false_negatives)
len_fp = len(false_positives)
print(jaccards)
print("Candidates:",len_candidates,", False negatives:",len_fn,", False positives:",len_fp)

[('g4pC_taskd.txt', 'orig_taske.txt', 0.3333333333333333)]
Candidates: 1 , False negatives: 0 , False positives: 0


In [46]:
b = 40
r = 25
B = 200
t = 0.1
banded_mat = bands(signature_1000,b,r)
hashed_docs = hash_vals(banded_mat,B)
banded_mat_orig = bands(signature_1000_orig,b,r)
hashed_docs_orig = hash_vals(banded_mat_orig,B)
test_orig = hashed_docs_orig[1].tolist()
orig_name = list(sh_5_orig.items())[1][0]

candidates, false_negatives, false_positives, jaccards = local_sh(hashed_docs,shing_5_docs,test_orig,t)
len_candidates = len(candidates)
len_fn = len(false_negatives)
len_fp = len(false_positives)
#print(jaccards)
print("Candidates:",len_candidates,", False negatives:",len_fn,", False positives:",len_fp)

Candidates: 8 , False negatives: 7 , False positives: 6
